In [1]:
import pandas as pd
import mysql.connector

In [2]:
connection = mysql.connector.connect(
    host= 'localhost',         
    user= 'root',             
    password= '123456', 
    database = 'day_51'     
)
cursor = connection.cursor()

In [3]:
cursor.execute("""
CREATE TABLE customers (
    id INTEGER PRIMARY KEY AUTO_INCREMENT,
    name VARCHAR(200) NOT NULL, 
    city VARCHAR(200) NOT NULL
);
""")

In [4]:
cursor.execute("""
INSERT INTO customers (id, name, city) VALUES
(1, 'Alice', 'Hanoi'),
(2, 'Bob', 'Ho Chi Minh'),
(3, 'Charlie', 'Hanoi');
""")

connection.commit()

In [5]:
cursor.execute("""
CREATE TABLE orders (
    id INTEGER PRIMARY KEY AUTO_INCREMENT,
    customer_id INTEGER NOT NULL,
    order_date DATE NOT NULL,
    total INTEGER NOT NULL,
    FOREIGN KEY (customer_id) REFERENCES customers(id)
);
""")

In [6]:
cursor.execute("""
INSERT INTO orders (id, customer_id, order_date, total) VALUES
(1, 1, '2024-01-10', 500),
(2, 1, '2024-03-01', 800),
(3, 2, '2024-02-20', 1200),
(4, 3, '2024-01-15', 200);
""")

connection.commit()

In [7]:
cursor.execute("""
CREATE TABLE products (
    id INTEGER PRIMARY KEY AUTO_INCREMENT,
    name VARCHAR(200) NOT NULL,
    price INTEGER NOT NULL
);
""")

In [8]:
cursor.execute("""
INSERT INTO products (id, name, price) VALUES
(1, 'Laptop', 1500),
(2, 'Mouse', 50),
(3, 'Keyboard', 100),
(4, 'Monitor', 300);
""")

connection.commit()

In [9]:
cursor.execute("""
CREATE TABLE order_items (
    id INTEGER PRIMARY KEY AUTO_INCREMENT,
    order_id INTEGER NOT NULL,
    product_id INTEGER NOT NULL,
    quantity INTEGER NOT NULL,
    FOREIGN KEY (order_id) REFERENCES orders(id),
    FOREIGN KEY (product_id) REFERENCES products(id)
);
""")

In [10]:
cursor.execute("""
INSERT INTO order_items (id, order_id, product_id, quantity) VALUES
(1, 1, 2, 2),
(2, 1, 3, 1),
(3, 2, 1, 1),
(4, 2, 4, 1),
(5, 3, 1, 1);
""")

connection.commit()

In [11]:
cursor.execute("""
CREATE TABLE employees (
    id INTEGER PRIMARY KEY AUTO_INCREMENT,
    name VARCHAR(200) NOT NULL,
    department VARCHAR(200) NOT NULL
);
""")

In [12]:
cursor.execute("""
INSERT INTO employees (id, name, department) VALUES
(1, 'David', 'Sales'),
(2, 'Emma', 'Support'),
(3, 'Frank', 'Sales');
""")

connection.commit()

In [13]:
cursor.execute("""
CREATE TABLE order_assignments (
    id INTEGER PRIMARY KEY AUTO_INCREMENT,
    order_id INTEGER NOT NULL,
    employee_id INTEGER NOT NULL,
    FOREIGN KEY (order_id) REFERENCES orders(id),
    FOREIGN KEY (employee_id) REFERENCES employees(id)
);
""")

In [14]:
cursor.execute("""
INSERT INTO order_assignments (id, order_id, employee_id) VALUES
(1, 1, 1),
(2, 2, 3),
(3, 3, 2);
""")

connection.commit()

### Bài 1

In [3]:
cursor.execute("""
CREATE VIEW customer_name_total AS
    SELECT c.name, SUM(o.total) AS total_spent
     FROM customers c
    LEFT JOIN orders o ON c.id = o.customer_id
    GROUP BY c.id;
""")

In [30]:
query = ("""
SELECT * 
 FROM customer_name_total
""")

data_1 = pd.read_sql_query(query, connection)
data_1

C:\Users\Apollonir\AppData\Local\Temp\ipykernel_21664\3869078944.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_1 = pd.read_sql_query(query, connection)


,name,total_spent
0,Alice,1300.0
1,Bob,1200.0
2,Charlie,200.0


### Bài 2

In [5]:
cursor.execute("""
CREATE VIEW product_price_greate_100 AS
    SELECT name, price
     FROM products 
    WHERE price > 100;
""")

In [29]:
query = ("""
SELECT * 
 FROM product_price_greate_100
""")

data_2 = pd.read_sql_query(query, connection)
data_2

C:\Users\Apollonir\AppData\Local\Temp\ipykernel_21664\3967403746.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_2 = pd.read_sql_query(query, connection)


,name,price
0,Laptop,1500
1,Monitor,300


### Bài 3

In [ ]:
cursor.execute("""
CREATE VIEW total_order_greate_1000 AS
    SELECT o.id, c.name, SUM(o.total) AS total_spent
     FROM (SELECT * FROM orders WHERE total > 1000) o 
    LEFT JOIN customers c ON c.id = o.customer_id
    GROUP BY o.id, c.name;
""")

In [28]:
query = ("""
SELECT * 
 FROM total_order_greate_1000
""")

data_3 = pd.read_sql_query(query, connection)
data_3

C:\Users\Apollonir\AppData\Local\Temp\ipykernel_21664\2483893748.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_3 = pd.read_sql_query(query, connection)


,id,name,total_spent
0,3,Bob,1200.0


### Bài 4

In [10]:
cursor.execute("""
CREATE VIEW employees_sale AS
    SELECT name
     FROM employees
    WHERE department = 'Sales';
""")

In [27]:
query = ("""
SELECT * 
 FROM employees_sale
""")

data_4 = pd.read_sql_query(query, connection)
data_4

C:\Users\Apollonir\AppData\Local\Temp\ipykernel_21664\3810287331.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_4 = pd.read_sql_query(query, connection)


,name
0,David
1,Frank


### Bài 5

In [12]:
cursor.execute("""
CREATE VIEW order_total AS
    SELECT o.id, SUM(o.quantity * p.price) AS total
     FROM order_items o
    LEFT JOIN products p ON p.id = o.product_id
    GROUP BY o.id;
""")

In [26]:
query = ("""
SELECT * 
 FROM order_total
""")

data_5 = pd.read_sql_query(query, connection)
data_5

C:\Users\Apollonir\AppData\Local\Temp\ipykernel_21664\1355874725.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_5 = pd.read_sql_query(query, connection)


,id,total
0,1,100.0
1,2,100.0
2,3,1500.0
3,4,300.0
4,5,1500.0


### Bài 6

In [20]:
cursor.execute("""
DROP VIEW IF EXISTS product_above_100;
""")

### Bài 7

In [21]:
cursor.execute("""
CREATE VIEW order_detail AS
    SELECT o1.id
        , p.name product_name
        , o1.quantity
        , o1.quantity * p.price total_price
        , o2.order_date
        , c.name customer_name
        , e.name employee_name
     FROM order_items o1
    LEFT JOIN orders o2 ON o1.order_id = o2.id
    LEFT JOIN products p ON o1.product_id = p.id
    LEFT JOIN customers c ON o2.customer_id = c.id
    LEFT JOIN order_assignments o3 on o1.order_id = o3.order_id
    LEFT JOIN employees e ON o3.employee_id = e.id;
""")

In [22]:
query = ("""
SELECT * 
 FROM order_detail
""")

data_7 = pd.read_sql_query(query, connection)
data_7

C:\Users\Apollonir\AppData\Local\Temp\ipykernel_21664\1933089731.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_7 = pd.read_sql_query(query, connection)


,id,product_name,quantity,total_price,order_date,customer_name,employee_name
0,1,Mouse,2,100,2024-01-10,Alice,David
1,2,Keyboard,1,100,2024-01-10,Alice,David
2,3,Laptop,1,1500,2024-03-01,Alice,Frank
3,4,Monitor,1,300,2024-03-01,Alice,Frank
4,5,Laptop,1,1500,2024-02-20,Bob,Emma


### Bài 8

In [24]:
cursor.execute("""
CREATE VIEW product_summary AS
    SELECT p.name product_name
        , SUM(o1.quantity) total_quantity
        , SUM(o1.quantity * p.price) total_price
     FROM order_items o1
    LEFT JOIN products p ON o1.product_id = p.id
    GROUP BY p.name;
""")

In [25]:
query = ("""
SELECT * 
 FROM product_summary
""")

data_8 = pd.read_sql_query(query, connection)
data_8

C:\Users\Apollonir\AppData\Local\Temp\ipykernel_21664\1721122982.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_8 = pd.read_sql_query(query, connection)


,product_name,total_quantity,total_price
0,Mouse,2.0,100.0
1,Keyboard,1.0,100.0
2,Laptop,2.0,3000.0
3,Monitor,1.0,300.0


### Bài 9

In [31]:
cursor.execute("""
CREATE VIEW order_hanoi AS
    SELECT o.id, c.name customer_name, o.total
     FROM orders o
    LEFT JOIN customers c ON o.customer_id = c.id
    WHERE c.city = 'Hanoi';
""")

In [32]:
query = ("""
SELECT * 
 FROM order_hanoi
""")

data_9 = pd.read_sql_query(query, connection)
data_9

C:\Users\Apollonir\AppData\Local\Temp\ipykernel_21664\3074755757.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_9 = pd.read_sql_query(query, connection)


,id,customer_name,total
0,1,Alice,500
1,2,Alice,800
2,4,Charlie,200


### Bài 10

In [ ]:
cursor.execute("""
CREATE VIEW order_hanoi AS
    SELECT c.name customer_name, SUM(o.total) total
     FROM orders o
    LEFT JOIN customers c ON o.customer_id = c.id
    GROUP BY c.name;
""")